In [1]:
import os
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=gpu'
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin/"
import sys
sys.path.append('/data/fs4/home/bradh/')
import theano
import numpy as np
import random
from copy import copy

import learningfunctions

import blocks
from blocks.bricks import Linear, Softmax, Softplus, NDimensionalSoftmax, BatchNormalizedMLP, \
                                Rectifier, Logistic, Tanh, MLP
from blocks.bricks.recurrent import GatedRecurrent, LSTM
from blocks.bricks.parallel import Fork
from blocks.initialization import Constant, IsotropicGaussian, Identity, Uniform
from blocks.bricks.cost import BinaryCrossEntropy, CategoricalCrossEntropy
from blocks.filter import VariableFilter
from blocks.roles import PARAMETER
from blocks.graph import ComputationGraph

import theano
from theano import tensor as T

Using gpu device 0: TITAN X (Pascal) (CNMeM is disabled, cuDNN 5105)
/opt/conda/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


# PUTTING IT ALL TOGETHER

In [2]:
def encoding_step(data_in, fork, rnn):
    def step(data_in):
        d1, d2 = fork.apply(data_in)
        result = rnn.apply(d1, d2)
        return result
    hEnc, _ = theano.scan(step, data_in)
    return hEnc

In [3]:
Xf = T.tensor4('Xf')
Xi = T.matrix('Xi')
Xs = T.tensor4('Xs')

dimInf = 50
dimIni = 60
dimIns = 30
dimComp = 10

wtstd = 0.1
rnnbias_init = Constant(0.0)
rnnwt_init = IsotropicGaussian(wtstd)
linewt_init = IsotropicGaussian(wtstd)
line_bias = Constant(1.0)
classifierWts = IsotropicGaussian(wtstd)
classifierBias = Constant(0.0001)

clippings = 1
learning_rate = 0.0001

fun = 0
actFunctList = [Rectifier(), Tanh()] #LeakyRectifier(leak=0.01)
actFunct = actFunctList[fun]

dimMultiplier = 2

### ENCODERS

In [4]:
#FUNCTION ENCODER
rnnf = GatedRecurrent(dim=dimComp, weights_init = rnnwt_init, biases_init = rnnbias_init, 
                      name = 'gru', activation=actFunct)
forkf = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dimInf, output_dims=[dimComp, dimComp * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

hEncf = encoding_step(Xf, forkf, rnnf)

rnn2f = GatedRecurrent(dim=dimComp, weights_init = rnnwt_init, biases_init = rnnbias_init, 
                       name = 'gru', activation=actFunct)
fork2f = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dimComp, output_dims=[dimComp, dimComp * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

hEnc2f = encoding_step(hEncf, fork2f, rnn2f)

rnn3f = GatedRecurrent(dim=dimComp, weights_init = rnnwt_init, biases_init = rnnbias_init, 
                       name = 'gru', activation=actFunct)
fork3f = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dimComp, output_dims=[dimComp, dimComp * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

hEnc3f = encoding_step(hEnc2f, fork3f, rnn3f)

#forkDf = Fork(output_names=['linear', 'gates'],
#            name='fork', input_dim=dimComp, output_dims=[dimInf, dimInf * dimMultiplier], 
#            weights_init = linewt_init, biases_init = line_bias)

h4decoderf = hEnc3f[:,-1,:,:].reshape((-1, 1,1,dimComp))
#h4reshapef, _ = forkDf.apply(h4decoderf)

In [5]:
#IMPORT ENCODER
encoder_bmlp = BatchNormalizedMLP( activations=[actFunct], 
           dims=[dimIni, dimComp],
           weights_init=classifierWts,
           biases_init= classifierBias)

encoder_output = encoder_bmlp.apply(Xi)

In [6]:
#SEMANTIC ENCODER
rnns = GatedRecurrent(dim=dimComp, weights_init = rnnwt_init, biases_init = rnnbias_init, 
                      name = 'gru', activation=actFunct)
forks = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dimIns, output_dims=[dimComp, dimComp * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

hEncs = encoding_step(Xs, forks, rnns)

rnn2s = GatedRecurrent(dim=dimComp, weights_init = rnnwt_init, biases_init = rnnbias_init,
                       name = 'gru', activation=actFunct)
fork2s = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dimComp, output_dims=[dimComp, dimComp * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

hEnc2s = encoding_step(hEncs, fork2s, rnn2s)

rnn3s = GatedRecurrent(dim=dimComp, weights_init = rnnwt_init, biases_init = rnnbias_init, 
                       name = 'gru', activation=actFunct)
fork3s = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dimComp, output_dims=[dimComp, dimComp * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

hEnc3s = encoding_step(hEnc2s, fork3s, rnn3s)

#forkDs = Fork(output_names=['linear', 'gates'],
#            name='fork', input_dim=dimComp, output_dims=[dimIns, dimIns * dimMultiplier], 
#            weights_init = linewt_init, biases_init = line_bias)

h4decoders = hEnc3s[:,-1,:,:].reshape((-1, 1,1,dimComp))
#h4reshapes, _ = forkDs.apply(h4decoders)



In [7]:
#TODO: add beam search
#TOTHINK: transform before the decoder or after

### JOINT VECTOR SPACE

In [8]:
#CONCATENATE
jvs = T.concatenate([T.squeeze(h4decoderf), encoder_output, T.squeeze(h4decoders)], axis = 1)
jvsReshaped = T.reshape(jvs, (-1, 1,1,dimComp*3))

### DECODERS

In [9]:
#FUNCTION DECODER
rnn4f = GatedRecurrent(dim=dimInf, weights_init = rnnwt_init, biases_init = rnnbias_init, 
                       name = 'gru', activation=actFunct)
fork4f = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dimComp*3, output_dims=[dimInf, dimInf * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

hDecf = encoding_step(jvsReshaped, fork4f, rnn4f)
targetsf = T.concatenate((hDecf, Xf[:,:-1, :,:]), axis=1)


predictedXf = T.exp(targetsf)/T.sum(T.exp(targetsf), axis=(3,2), keepdims=True)
costf = T.mean(T.sum(T.nnet.categorical_crossentropy(predictedXf, Xf), axis = 1))

test = theano.function([Xf], hEncf, allow_input_downcast=True)

In [14]:
fakeXf = np.random.rand(13, 30, 1, dimInf)
fakeXi = np.random.rand(13, dimIni)
fakeXs = np.random.rand(13, 100, 1, dimIns)

test(fakeXf)

In [10]:
#IMPORT DECODER
decoder_bmlp = BatchNormalizedMLP( activations=[actFunct], 
           dims=[dimComp*3, dimIni],
           weights_init=classifierWts,
           biases_init=classifierBias )

decoder_output = decoder_bmlp.apply(jvs)

predictedXi = T.exp(decoder_output)/T.sum(T.exp(decoder_output), axis = 1, keepdims=True)
costi = T.mean(CategoricalCrossEntropy().apply(Xi, predictedXi))

In [11]:
#SEMANTIC DECODER
rnn4s = GatedRecurrent(dim=dimIns, weights_init = rnnwt_init, biases_init = rnnbias_init, 
                       name = 'gru', activation=actFunct)
fork4s = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dimComp*3, output_dims=[dimIns, dimIns * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

hDecs = encoding_step(jvsReshaped, fork4s, rnn4s)
targetss = T.concatenate((hDecs, Xs[:,:-1, :,:]), axis=1)


predictedXs = T.exp(targetss)/T.sum(T.exp(targetss), axis=(3,2), keepdims=True)
costs = T.mean(T.sum(T.nnet.categorical_crossentropy(predictedXs, Xs), axis = 1))

In [12]:
#INTIALIZE
fork = [forkf, fork2f, fork3f, fork4f, forks, fork2s, fork3s, fork4s]
rnn = [rnnf, rnn2f, rnn3f, rnn4f, rnns, rnn2s, rnn3s, rnn4s]
mlp = [encoder_bmlp, decoder_bmlp]

for finit in fork:
    finit.initialize()
for rinit in rnn:
    rinit.initialize()
for minit in mlp:
    minit.initialize()

In [13]:
cost = costf+costi+costs
cg = ComputationGraph([cost])
params = VariableFilter(roles = [PARAMETER])(cg.variables)

learning = learningfunctions.Learning(cost,params,learning_rate,l1=0.,l2=0.,maxnorm=0.,c=clippings)
updates = learning.Adam() 

print('compiling graph you talented soul')
train = theano.function([Xf,Xi,Xs], cost, updates=updates, allow_input_downcast=True)
predict = theano.function([Xf,Xi,Xs], [predictedXf, predictedXi, predictedXs], allow_input_downcast=True)
print('finished compiling')

compiling graph you talented soul
finished compiling


In [15]:
predict(fakeXf, fakeXi, fakeXs)

[array([[[[ 0.03048947,  0.01547793,  0.0117109 , ...,  0.02137607,
            0.01571949,  0.03888531]],
 
         [[ 0.01336728,  0.0158511 ,  0.01565423, ...,  0.01373032,
            0.01990004,  0.02536408]],
 
         [[ 0.01279809,  0.01629212,  0.01187036, ...,  0.01796917,
            0.0194895 ,  0.01943433]],
 
         ..., 
         [[ 0.0123929 ,  0.02321035,  0.02288632, ...,  0.01349519,
            0.01841797,  0.01474056]],
 
         [[ 0.01795548,  0.02480968,  0.01189146, ...,  0.01939003,
            0.02043458,  0.02170853]],
 
         [[ 0.03086285,  0.02438878,  0.01831656, ...,  0.03069382,
            0.0126109 ,  0.02171484]]],
 
 
        [[[ 0.0307118 ,  0.01511004,  0.01143822, ...,  0.02172776,
            0.01649292,  0.04105354]],
 
         [[ 0.01584915,  0.02096317,  0.02028376, ...,  0.01433983,
            0.01677924,  0.01198772]],
 
         [[ 0.01236266,  0.01304613,  0.01977151, ...,  0.02040976,
            0.03123224,  0.01624589]],
 
 

In [8]:
from redbaron import RedBaron

In [3]:
with open('/data/fs4/home/bradh/learningfunctions.py', 'r') as f:
    testscript = f.readlines()

In [5]:
teststring = ''.join(testscript)

In [9]:
red = RedBaron(teststring)

In [46]:
red.node_list[7].node_list[1].node_list[1].target.dumps()

'self.cost'

In [19]:
# class = 'ClassNode', def = 'DefNode', vars
#funnames = []
for fun in red.find_all('ClassNode'):
    funnames.append(fun.name)

In [53]:
#var names
red.find_all('AssignmentNode')[0].type

'assignment'

In [64]:
red = RedBaron(teststring)
thirdKing = []

def parseTree(red, thirdKing):
    if not hasattr(red, 'node_list'):
        return thirdKing
    
    for node in red.node_list:
        #print(node.type)
        if node.type == 'class':
            classNode = [{'type': 'class', 'name': node.name}]
            thirdKing.append(parseTree(node, classNode))
            
        elif node.type == 'def':
            defNode = [{'type': 'def', 'name': node.name}]
            thirdKing.append(parseTree(node, defNode))
            
        elif node.type == 'assignment':
            varNode = [{'type': 'var', 'name': node.target.dumps()}]
            thirdKing.append(parseTree(node, varNode))
        
    return thirdKing
            

In [67]:
parsed = parseTree(red, [])